In [3]:
import numpy as np
import os
import pandas as pd
import pprint
from sklearn.preprocessing import StandardScaler
import sys

sys.path.insert(0, os.path.join(os.getcwd(), "../"))

from model_utils import *

## Get Label Distributions

In [4]:
PATH_TO_DATA_SPLITS = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/data_splits"
PATH_TO_CELLPROFILER_SPLITS_OUTPUT = os.path.join(PATH_TO_DATA_SPLITS, "custom_splits/cellprofiler/stardist_patch_num=4")

PATH_TO_TRAIN = os.path.join(PATH_TO_CELLPROFILER_SPLITS_OUTPUT, "train.csv")
PATH_TO_VAL = os.path.join(PATH_TO_CELLPROFILER_SPLITS_OUTPUT, "val.csv")
PATH_TO_TEST = os.path.join(PATH_TO_CELLPROFILER_SPLITS_OUTPUT, "test.csv")

In [ ]:
train_features_df = pd.read_csv(PATH_TO_TRAIN)
train_features_df["patient_id"] = train_features_df["Image"].apply(lambda im : im.split("-")[0])

print(train_features_df.shape)
train_features_df["label"].value_counts()

In [ ]:
val_features_df = pd.read_csv(PATH_TO_VAL)
val_features_df["patient_id"] = val_features_df["Image"].apply(lambda im : im.split("-")[0])

print(val_features_df.shape)
val_features_df["label"].value_counts()

In [ ]:
test_features_df = pd.read_csv(PATH_TO_TEST)
test_features_df["patient_id"] = test_features_df["Image"].apply(lambda im : im.split("-")[0])

print(test_features_df.shape)
test_features_df["label"].value_counts()

In [8]:
columns = list(train_features_df.columns)
# Update this list based on the specific features.
columns_to_remove = ["Image", "patient_id", "tma_id", "label", "split", "NormalizedMoment", "count",
                     "Unnamed", "nucleiObjectNumber", "cytoplasmObjectNumber"]
for column_name in columns_to_remove:
    columns = list(filter(lambda col: not(column_name in col), columns))
all_features = columns

## Get Features of Type

In [ ]:
feature_types = set(list(map(lambda s : s.split("_")[0], all_features)))
feature_types

In [ ]:
def get_features_of_type(all_features, feature_type):
    return list(filter(lambda s : s.split("_")[0] == feature_type, all_features))

features_of_type = get_features_of_type(all_features, "nucleiLocation")
features_of_type

In [11]:
def get_area_shape_features(feature_type):
    assert(feature_type == "nucleiAreaShape" or feature_type == "cytoplasmAreaShape")
    all_area_shape_features = get_features_of_type(all_features, feature_type)
    # These are spatial features, so we don't include them for Section 1.1 experiments.
    to_remove = ["BoundingBoxMaximum", "BoundingBoxMinimum", "Center", "CentralMoment", "SpatialMoment"]
    area_shape_features = []
    for feature in all_area_shape_features:
        if feature.split("_")[1] not in to_remove:
            area_shape_features.append(feature)
    return area_shape_features

def get_location_features(feature_type):
    assert(feature_type == "nucleiLocation" or feature_type == "cytoplasmLocation")
    all_location_features = get_features_of_type(all_features, feature_type)
    # These are spatial features, so we don't include them for Section 1.1 experiments.
    to_remove = ["Center"]
    location_features = []
    for feature in all_location_features:
        if feature.split("_")[1] not in to_remove:
            location_features.append(feature)
    return location_features

def get_nuclei_morphological_features(all_features):
    area_shape_features = get_area_shape_features("nucleiAreaShape")
    number_features = get_features_of_type(all_features, "nucleiNumber")
    return area_shape_features + number_features

def get_nuclei_intensity_features(all_features):
    area_shape_features = get_area_shape_features("nucleiAreaShape")
    number_features = get_features_of_type(all_features, "nucleiNumber")
    location_features = get_location_features("nucleiLocation")
    intensity_features = get_features_of_type(all_features, "nucleiIntensity")
    return area_shape_features + number_features + location_features + intensity_features

def get_nuclei_cytoplasm_morphological_features(all_features):
    n_area_shape_features = get_features_of_type(all_features, "nucleiAreaShape")
    n_number_features = get_features_of_type(all_features, "nucleiNumber")
    n_children_features = get_features_of_type(all_features, "nucleiChildren")
    
    c_area_shape_features = get_features_of_type(all_features, "cytoplasmAreaShape")
    c_number_features = get_features_of_type(all_features, "cytoplasmNumber")
    c_parent_features = get_features_of_type(all_features, "cytoplasmParent")
    
    return (n_area_shape_features + n_number_features + n_children_features + 
            c_area_shape_features + c_number_features + c_parent_features)

nuclei_features = get_nuclei_morphological_features(all_features)
nuclei_intensity_features = get_nuclei_intensity_features(all_features)
nuclei_cyto_features = get_nuclei_cytoplasm_morphological_features(all_features)

## Read Additional Stains

In [14]:
from pandas import read_excel

In [15]:
PATH_TO_RAW_DATA = "/deep/group/aihc-bootcamp-fall2021/lymphoma/raw"
PATH_TO_MASTER_KEY = os.path.join(PATH_TO_RAW_DATA, "Guatemala Project Data vFINAL.xlsx")
PATH_TO_CLPA_FILE = os.path.join(PATH_TO_RAW_DATA, "CLPA Diagnostic Bin.xlsx")
CASE = "CASE"
WHO_DIAGNOSIS = "2017 WHO DIAGNOSIS"
CLPA_DIAGNOSIS = "CLPA Diagnostic Bin"
LABEL = "label"
TMA_ID = "TMA ID"

PATH_TO_DATA_SPLITS = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/data_splits"
PATH_TO_SPLITS_OUTPUT = os.path.join(PATH_TO_DATA_SPLITS, "custom_splits/stains")

pd.set_option("display.max_rows", 400)
pd.set_option("display.max_columns", 100)

In [16]:
def get_df_from_set(set_index, set_name):
    # Read the diagnoses from sheet 'set_name' in the Master Key file and store
    # them in the dataframe.
    df = read_excel(PATH_TO_MASTER_KEY, sheet_name=set_name,
                    engine='openpyxl')
    df[TMA_ID] = set_index
    return df

In [17]:
all_sheet_names = ["Set(1)", "Set(2)", "Set(3)", "Set (4)", "Set (5)", "Set (6)", "Set (7)", "Set (8)"]
# Create a dataframe containing the diagnoses for each patient ID in all
# the TMAs inside the Master Key.
ihc_df = pd.concat([get_df_from_set(set_index + 1, set_name) for set_index, set_name in enumerate(all_sheet_names)])

In [18]:
ihc_df = ihc_df[ihc_df[CASE].str.contains("E0", na = False)]

In [19]:
ihc_df["CASE"] = ihc_df["CASE"].apply(lambda case : case.replace(" ", ""))
ihc_df = ihc_df.fillna(-1)

## Add stains to dataframes

In [20]:
STAIN_LIST = ['CD20', 'CD10', 'PAX5', 'EBV ISH', 'BCL1', 'BCL6', 'TIA', 'FISH - MYC']

In [21]:
def add_stain_to_df(df, stain):
    stain_map = ihc_df.set_index('CASE')[stain].to_dict()
    stain_list = []
    for patient_id in df["patient_id"]:
        case_key = patient_id.split("_")[2].replace(" ", "")
        if case_key in stain_map:
            stain_list.append(stain_map[case_key])
        else:
            print(case_key)
            stain_list.append("NaN")
    df[stain] = stain_list
    return df

def add_stains_to_df(df):
    value_map = {"3 (dim)": 3, "3(dim)": 3, "3 (vari)": 3, "3 (dim subset?)": 3, "3 (mod)": 3, "3 (variable)": 3, "3 (bright)": 1,
             "3 (strong)": 3, "3 (100%)": 3, "3 (cyto)": 3,
             "2 (dim)": 2, "2 (vari)": 2,
             "?": 0, "N": 0, "E": 0, "NN": 0, "sprouts": 0, float('nan') : -1}
    for stain in STAIN_LIST:
        df = add_stain_to_df(df, stain)
        df[stain] = df[stain].apply(lambda value : value_map[value] if value in value_map else value)
    return df

train_features_df = add_stains_to_df(train_features_df)
val_features_df = add_stains_to_df(val_features_df)
test_features_df = add_stains_to_df(test_features_df)
all_features.extend(STAIN_LIST)

In [20]:
all_features[-15:]

['cytoplasmParent_cells_kurtosis',
 'cytoplasmParent_cells_iqr',
 'cytoplasmParent_nuclei_mean',
 'cytoplasmParent_nuclei_std',
 'cytoplasmParent_nuclei_skew',
 'cytoplasmParent_nuclei_kurtosis',
 'cytoplasmParent_nuclei_iqr',
 'CD20',
 'CD10',
 'PAX5',
 'EBV ISH',
 'BCL1',
 'BCL6',
 'TIA',
 'FISH - MYC']

## Model Utilities

In [36]:
pp = pprint.PrettyPrinter(indent=4)
LABEL_DIST = [0.448980, 0.162065, 0.102041, 0.088836, 0.074430, 0.058824, 0.044418, 0.018007, 0.002401]

label_mapping = {0 : 0, # DLBCL -> DLBCL, Agg BCL (0)
                 1 : 1, # HL -> HL (1)
                 2 : 0, # Agg BCL -> DLBCL, Agg BCL (0)
                 3 : 2, # FL -> CLL, FL, MZL (2)
                 4 : 3, # MCL -> MCL (3)
                 5 : 2, # MZL -> CLL, FL, MZL (2)
                 6 : 4, # TCL -> TCL (4)
                 7 : 4, # NKTCL -> NKTCL (4)
                 8 : 5}

def group_labels(label):
    return label_mapping[label]

def get_processed_df_splits(feature_cols):
    train_geo_features_df = train_features_df[feature_cols + ["patient_id", "label", "count"]].dropna().reset_index(drop=True)
    val_geo_features_df = val_features_df[feature_cols + ["patient_id", "label", "count"]].dropna().reset_index(drop=True)
    test_geo_features_df = test_features_df[feature_cols + ["patient_id", "label", "count"]].dropna().reset_index(drop=True)
    
    train_geo_features_df["label"] = train_geo_features_df["label"].apply(group_labels)
    val_geo_features_df["label"] = val_geo_features_df["label"].apply(group_labels)
    test_geo_features_df["label"] = test_geo_features_df["label"].apply(group_labels)
    
    return (train_geo_features_df, val_geo_features_df, test_geo_features_df)

def get_splits(feature_cols, enable_dlbcl_classification=False, enable_normalization=True):
    (train_geo_features_df, val_geo_features_df, test_geo_features_df) = get_processed_df_splits(feature_cols)
    
    X_train = train_geo_features_df[feature_cols].astype(np.float32)
    y_train = train_geo_features_df["label"]
    X_val = val_geo_features_df[feature_cols].astype(np.float32)
    y_val = val_geo_features_df["label"]
    X_test = test_geo_features_df[feature_cols].astype(np.float32)
    y_test = test_geo_features_df["label"]
    
    if enable_dlbcl_classification:
        y_train = y_train.apply(lambda l : 0 if l == 0 else 1)
        y_val = y_val.apply(lambda l : 0 if l == 0 else 1)
        y_test = y_test.apply(lambda l : 0 if l == 0 else 1)
    
    if enable_normalization:
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)
    return (X_train, y_train, X_val, y_val, X_test, y_test, scaler)

def get_top_3_accuracy(top_3_test_preds, y_test):
    successes = 0
    for i in range(y_test.shape[0]):
        if y_test[i] in top_3_test_preds[i][0]:
            successes += 1
    return float(successes)/ y_test.shape[0]

def get_core_metrics(features_df, preds_patch, enable_dlbcl_classification=False):
    features_df["preds_patch"] = preds_patch
    y_core = features_df.groupby("patient_id")["label"].agg(pd.Series.mode)
    if enable_dlbcl_classification:
        y_core = y_core.apply(lambda l : 0 if l == 0 else 1)
    preds_core = features_df.groupby("patient_id")["preds_patch"].agg(lambda x: pd.Series.mode(x)[0])
    accuracy = compute_accuracy(preds_core, y_core)
    return (y_core, preds_core, accuracy)

def get_sgd_pred_probs(model, X):
    return model.predict_proba(X)

def get_lgb_pred_probs(model, X):
    return softmax(model.predict(X))

def get_mean_probability(pred_probs):
    return np.argmax(pd.Series.mean(pred_probs, axis=0))

def get_core_metrics_mean(features_df, pred_probs):
    classes = range(num_classes)
    pred_probs_df = pd.DataFrame(pred_probs, columns = classes)
    pred_probs_df["patient_id"] = features_df["patient_id"]
    pred_prob_df_aggregated = pred_probs_df.groupby("patient_id").aggregate(pd.DataFrame.mean)
    y_core = features_df.groupby("patient_id")["label"].agg(pd.Series.mode)
    preds_core = pred_prob_df_aggregated.idxmax(axis=1)
    accuracy = compute_accuracy(preds_core, y_core)
    return (y_core, preds_core, accuracy)

def accuracy_eval_metric(preds, dtrain):
    labels = dtrain.label
    preds = preds.reshape(num_classes, -1).T
    preds = preds.argmax(axis=1)
    accuracy = accuracy_score(labels, preds)
    return 'accuracy', accuracy, False
